In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calc_mutual_inductance(coil_1, coil_2, d, po=0, fi=0):
    mu0 = 4 * np.pi * 10 ** (-7)
    mutual_inductance = np.ones((len(coil_1), len(coil_2)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    for ri in range(len(coil_1)):
        for rj in range(len(coil_2)):
            M = 0
            for n in range(N):
                for k in range(K):
                    xk_xn = po + coil_1[ri] * np.cos(df2 * k) * np.cos(fi) - coil_2[rj] * np.cos(df1 * n)
                    yk_yn = coil_1[ri] * np.sin(df2 * k) * np.cos(fi) - coil_2[rj] * np.sin(df1 * n)
                    zk_zn = d + coil_1[ri] * np.cos(df2 * k) * np.sin(fi)
                    r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                    M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
            M *= mu0 * coil_1[ri] * coil_2[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    return np.sum(mutual_inductance)

def calc_self_inductance(coil, thin=0.001):
    mu0 = 4 * np.pi * 10 ** (-7)
    L = np.sum(mu0 * coil * (np.log(8 * coil / thin) - 7 / 4 + (thin ** 2) / (8 * coil ** 2) * (np.log(8 * coil / thin) + 1 / 3)))
    mutual_inductance = np.ones((len(coil), len(coil)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    d = 0
    po = 0
    fi = 0
    for ri in range(len(coil)):
        for rj in range(len(coil)):
            M = 0
            if ri != rj:
                for n in range(N):
                    for k in range(K):
                        xk_xn = po + coil[ri] * np.cos(df2 * k) * np.cos(fi) - coil[rj] * np.cos(df1 * n)
                        yk_yn = coil[ri] * np.sin(df2 * k) * np.cos(fi) - coil[rj] * np.sin(df1 * n)
                        zk_zn = d + coil[ri] * np.cos(df2 * k) * np.sin(fi)
                        r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                        M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
                M *= mu0 * coil[ri] * coil[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    M = np.sum(mutual_inductance)
    L += M
    return L

def calc_coupling_coefficient(coil_1, coil_2, d, po=0, fi=0):
    M = calc_mutual_inductance(coil_1, coil_2, d, po=po, fi=fi)
    L1 = calc_self_inductance(coil_1)
    L2 = calc_self_inductance(coil_2)
    k = M / (L1 * L2) ** 0.5
    return k

def mutation(start, finish, x, r):
    x = np.random.uniform(low = start if x - r < start else x - r,
                          high = finish if x + r > finish else x + r)
    return x

# между витками одной катушки не должно быть пересечений
def test_coils(coil, thin):
    # между витками 2 и 3 катушки есть пересечение
    if coil[1] > coil[2]:
        if (coil[1] - coil[2] - 2 * thin) < 0:
            return False
    elif coil[2] > coil[1]:
        if (coil[2] - coil[1] - 2 * thin) < 0:
            return False
    return True

In [3]:
def steepest_ascent_hill_climbing(coil_1, thin, d):
    good_mutation = []
    bad_mutation = []
    all_mutation = []
    
    start = 0.03 + 2 * thin
    finish = 0.09 - 2 * thin
    coil_2 = np.array([0.03, start, finish, 0.09])
    
    Q = 2
    band = 0.005
    thr_k = 1e-4
    
    print("Начальное значение катушки:", coil_2)
    i = 0
    while True:
        i += 1
        fit_k = calc_coupling_coefficient(coil_1, coil_2, d)
        num_q = []
        for q in range(Q):
            coil_2q = coil_2.copy()
            coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
            # проверка на пересечение витков внутри катушки
            while not test_coils(coil_2q, thin):
                print(f"Есть пересечения внутри катушки! {coil_2q}")
                coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
            fit_kq = calc_coupling_coefficient(coil_1, coil_2q, d)
            num_q.append((fit_kq, coil_2q))
        num_q = sorted(num_q, key=lambda x: x[0], reverse=True)[0]
        if num_q[0] > fit_k:
            fit_k = num_q[0].copy()
            coil_2 = num_q[1].copy()
            good_mutation.append(num_q)
        else:
            bad_mutation.append(num_q)
        all_mutation.append(num_q)
        print(f"iter: {i} k = {fit_k}, катушка = {coil_2}")
        if len(good_mutation) >= 2:
            if (good_mutation[-1][0] - good_mutation[-2][0]) / thr_k < 1:
                break
        if i == 100:
            print("Алгоритм сделал 100 итераций! Останавливаю алгоритм")
            break
    return good_mutation[-1][0], good_mutation[-1][1], len(good_mutation), len(bad_mutation), len(all_mutation)



In [4]:
info_sahc = []

coil_1 = np.linspace(0.028, 0.07, 4)
thin = 0.001
d = 0.005
N = 10

In [5]:
for i in range(N):
    print(f"{i} итерация запуска алгоритма steepest ascent hill climbing:", end='\n\n')
    info_sahc.append(steepest_ascent_hill_climbing(coil_1, thin, d))

0 итерация запуска алгоритма steepest ascent hill climbing:

Начальное значение катушки: [0.03  0.032 0.088 0.09 ]
iter: 1 k = 0.45216494007237806, катушка = [0.03       0.03527833 0.088      0.09      ]
iter: 2 k = 0.4998122109366232, катушка = [0.03       0.03527833 0.08358411 0.09      ]
iter: 3 k = 0.5017242005400256, катушка = [0.03       0.03568159 0.08358411 0.09      ]
iter: 4 k = 0.5167697237047042, катушка = [0.03       0.03568159 0.0817269  0.09      ]
iter: 5 k = 0.5361544756044841, катушка = [0.03       0.03964604 0.0817269  0.09      ]
iter: 6 k = 0.5632194093724078, катушка = [0.03       0.03964604 0.07838739 0.09      ]
iter: 7 k = 0.5632194093724078, катушка = [0.03       0.03964604 0.07838739 0.09      ]
iter: 8 k = 0.5841914666252613, катушка = [0.03       0.03964604 0.07589875 0.09      ]
iter: 9 k = 0.5894396897721021, катушка = [0.03       0.04086123 0.07589875 0.09      ]
iter: 10 k = 0.5967622276888594, катушка = [0.03       0.04349022 0.07589875 0.09      ]
ite

iter: 16 k = 0.6446998235669963, катушка = [0.03       0.04489279 0.06843532 0.09      ]
iter: 17 k = 0.6446998235669963, катушка = [0.03       0.04489279 0.06843532 0.09      ]
iter: 18 k = 0.6452535197302777, катушка = [0.03       0.04489279 0.06577103 0.09      ]
iter: 19 k = 0.6459629171379698, катушка = [0.03       0.04698028 0.06577103 0.09      ]
iter: 20 k = 0.6459629171379698, катушка = [0.03       0.04698028 0.06577103 0.09      ]
iter: 21 k = 0.6459629171379698, катушка = [0.03       0.04698028 0.06577103 0.09      ]
iter: 22 k = 0.6482292796188175, катушка = [0.03       0.05003516 0.06577103 0.09      ]
iter: 23 k = 0.6493209908572904, катушка = [0.03       0.0509181  0.06577103 0.09      ]
iter: 24 k = 0.6530981518383305, катушка = [0.03       0.05409945 0.06577103 0.09      ]
iter: 25 k = 0.6530981518383305, катушка = [0.03       0.05409945 0.06577103 0.09      ]
iter: 26 k = 0.6530981518383305, катушка = [0.03       0.05409945 0.06577103 0.09      ]
iter: 27 k = 0.653098

iter: 1 k = 0.4540972184211221, катушка = [0.03       0.03571057 0.088      0.09      ]
iter: 2 k = 0.4610024468622931, катушка = [0.03       0.03721515 0.088      0.09      ]
iter: 3 k = 0.49716720894454314, катушка = [0.03       0.03721515 0.08497771 0.09      ]
iter: 4 k = 0.5237039777096685, катушка = [0.03       0.03721515 0.08179336 0.09      ]
iter: 5 k = 0.5410641910228265, катушка = [0.03       0.04088803 0.08179336 0.09      ]
iter: 6 k = 0.5755982659358275, катушка = [0.03       0.04088803 0.07754705 0.09      ]
iter: 7 k = 0.5755982659358275, катушка = [0.03       0.04088803 0.07754705 0.09      ]
iter: 8 k = 0.5789380709561056, катушка = [0.03       0.0418264  0.07754705 0.09      ]
iter: 9 k = 0.6073645782411896, катушка = [0.03       0.0418264  0.07419646 0.09      ]
iter: 10 k = 0.6330470204145758, катушка = [0.03       0.0418264  0.07066157 0.09      ]
iter: 11 k = 0.6410371908407496, катушка = [0.03       0.0418264  0.06607683 0.09      ]
iter: 12 k = 0.64557432323816

iter: 34 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 35 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 36 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 37 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 38 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 39 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 40 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 41 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 42 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 43 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 44 k = 0.6550211790067536, катушка = [0.03       0.05545387 0.06810548 0.09      ]
iter: 45 k = 0.655092

iter: 80 k = 0.6486360183384429, катушка = [0.03       0.04401334 0.05785087 0.09      ]
iter: 81 k = 0.6486360183384429, катушка = [0.03       0.04401334 0.05785087 0.09      ]
iter: 82 k = 0.6486360183384429, катушка = [0.03       0.04401334 0.05785087 0.09      ]
iter: 83 k = 0.6486360183384429, катушка = [0.03       0.04401334 0.05785087 0.09      ]
iter: 84 k = 0.6486360183384429, катушка = [0.03       0.04401334 0.05785087 0.09      ]
iter: 85 k = 0.6486360183384429, катушка = [0.03       0.04401334 0.05785087 0.09      ]
iter: 86 k = 0.6486360183384429, катушка = [0.03       0.04401334 0.05785087 0.09      ]
iter: 87 k = 0.6486360183384429, катушка = [0.03       0.04401334 0.05785087 0.09      ]
iter: 88 k = 0.6486360183384429, катушка = [0.03       0.04401334 0.05785087 0.09      ]
iter: 89 k = 0.648658313184242, катушка = [0.03       0.0442035  0.05785087 0.09      ]
9 итерация запуска алгоритма steepest ascent hill climbing:

Начальное значение катушки: [0.03  0.032 0.088 0.0

In [6]:
sorted(info_sahc, key=lambda x: x[0], reverse=True)

[(0.6551468247049117,
  array([0.03      , 0.05503378, 0.06807737, 0.09      ]),
  23,
  15,
  38),
 (0.6551168442143316,
  array([0.03      , 0.05499233, 0.06784345, 0.09      ]),
  17,
  15,
  32),
 (0.6550979176724642,
  array([0.03      , 0.05487269, 0.06778405, 0.09      ]),
  21,
  17,
  38),
 (0.6550921791039221,
  array([0.03      , 0.05529939, 0.06810548, 0.09      ]),
  18,
  27,
  45),
 (0.6550785175100027,
  array([0.03      , 0.05484338, 0.06773165, 0.09      ]),
  23,
  44,
  67),
 (0.6549882055394115,
  array([0.03      , 0.05481589, 0.06752718, 0.09      ]),
  15,
  1,
  16),
 (0.653714639906202,
  array([0.03      , 0.05466848, 0.06948574, 0.09      ]),
  22,
  16,
  38),
 (0.648658313184242,
  array([0.03      , 0.0442035 , 0.05785087, 0.09      ]),
  15,
  74,
  89),
 (0.6479588622448323,
  array([0.03      , 0.0491634 , 0.06761683, 0.09      ]),
  19,
  7,
  26),
 (0.6433873799341542,
  array([0.03      , 0.04286996, 0.06707273, 0.09      ]),
  14,
  5,
  19)]

In [25]:
print(f"Лучший коэффициент связи {info_sahc[0][0]} у катушки {info_sahc[0][1]}")
print(f"Среднее количество итераций {sum([i[4] for i in info_sahc]) / len(info_sahc)}")
print(f"Среднее количество полезных мутаций {sum([i[2] for i in info_sahc]) / len(info_sahc)}")
print(f"Среднее количество бесполезных мутаций {sum([i[3] for i in info_sahc]) / len(info_sahc)}")

Лучший коэффициент связи 0.6550979176724642 у катушки [0.03       0.05487269 0.06778405 0.09      ]
Среднее количество итераций 40.8
Среднее количество полезных мутаций 18.7
Среднее количество бесполезных мутаций 22.1
